In [ ]:
# for submission to ICLR 2026

import numpy as np

# task definitions
from utils.schema import df_param_SG_dict, df_param_DG_dict, df_param_Cauchy_dict, df_param_StudentT_dict

# for information gap calculation
import utils.functions as f
from information_gap.generative_model import TaskPrior, GaussianLikelihoodModel
from information_gap.information_gap import LikelihoodCodingInformationGapCalculator, PosteriorCodingInformationGapCalculator

# information gap calculation

In [ ]:
# define thetas
theta_start = -90
theta_end = 90
bins_per_degree = 1
theta_num = int((theta_end - theta_start) * bins_per_degree) + 1
thetas = np.linspace(theta_start, theta_end, theta_num)
print(f'thetas: {thetas.shape} {thetas[0]} to {thetas[-1]}')

# define likelihood
equivalent_sigma = 15.0

# define tolerance for identical posteriors
tolerance_identical_posteriors = 1e-5

In [ ]:
# Define task priors for two tasks A and B

################################################
# single Gaussian priors
task_type = 'SG'  # single Gaussian
df_task_param_dict = df_param_SG_dict
task_name = '-10_10_10_10'

In [ ]:
# define task priors
df_task_params = df_task_param_dict[task_name]
p_task_A = df_task_params['p_task'].iloc[0]
p_task_B = df_task_params['p_task'].iloc[1]

if task_type == 'SG':
    prior_A = TaskPrior(
        name=f'A:{task_type}_{df_task_params["mu"].iloc[0]}_{df_task_params["sigma"].iloc[0]}',
        thetas=thetas,
        distribution=f.get_Gaussian_pdf(
            mu=df_task_params["mu"].iloc[0], 
            sigma=df_task_params["sigma"].iloc[0], 
            thetas=thetas
        )
    )
    prior_B = TaskPrior(
        name=f'B:{task_type}_{df_task_params["mu"].iloc[1]}_{df_task_params["sigma"].iloc[1]}',
        thetas=thetas,
        distribution=f.get_Gaussian_pdf(
            mu=df_task_params["mu"].iloc[1], 
            sigma=df_task_params["sigma"].iloc[1], 
            thetas=thetas
        )
    )

elif task_type == 'DG':
    prior_A = TaskPrior(
        name=f'A:{task_type}_{df_task_params["mu_1"].iloc[0]}_{df_task_params["sigma_1"].iloc[0]}_{df_task_params["mu_2"].iloc[0]}_{df_task_params["sigma_2"].iloc[0]}',
        thetas=thetas,
        distribution=f.get_DoubleGaussians_pdf(
            mu_1=df_task_params["mu_1"].iloc[0], 
            sigma_1=df_task_params["sigma_1"].iloc[0], 
            mu_2=df_task_params["mu_2"].iloc[0], 
            sigma_2=df_task_params["sigma_2"].iloc[0], 
            thetas=thetas
        )
    )
    prior_B = TaskPrior(
        name=f'B:{task_type}_{df_task_params["mu_1"].iloc[1]}_{df_task_params["sigma_1"].iloc[1]}_{df_task_params["mu_2"].iloc[1]}_{df_task_params["sigma_2"].iloc[1]}',
        thetas=thetas,
        distribution=f.get_DoubleGaussians_pdf(
            mu_1=df_task_params["mu_1"].iloc[1], 
            sigma_1=df_task_params["sigma_1"].iloc[1], 
            mu_2=df_task_params["mu_2"].iloc[1], 
            sigma_2=df_task_params["sigma_2"].iloc[1], 
            thetas=thetas
        )
    )

elif task_type == 'Cauchy':
    prior_A = TaskPrior(
        name=f'A:{task_type}_{df_task_params["mu"].iloc[0]}_{df_task_params["gamma"].iloc[0]}',
        thetas=thetas,
        distribution=f.get_Cauchy_pdf(
            mu=df_task_params["mu"].iloc[0], 
            gamma=df_task_params["gamma"].iloc[0], 
            thetas=thetas
        )
    )
    prior_B = TaskPrior(
        name=f'B:{task_type}_{df_task_params["mu"].iloc[1]}_{df_task_params["gamma"].iloc[1]}',
        thetas=thetas,
        distribution=f.get_Cauchy_pdf(
            mu=df_task_params["mu"].iloc[1], 
            gamma=df_task_params["gamma"].iloc[1], 
            thetas=thetas
        )
    )

elif task_type == 'StudentT':
    prior_A = TaskPrior(
        name=f'A:{task_type}_{df_task_params["mu"].iloc[0]}_{df_task_params["df"].iloc[0]}_{df_task_params["sigma"].iloc[0]}',
        thetas=thetas,
        distribution=f.get_StudentT_pdf(
            mu=df_task_params["mu"].iloc[0], 
            df=df_task_params["df"].iloc[0], 
            sigma=df_task_params["sigma"].iloc[0],
            thetas=thetas
        )
    )
    prior_B = TaskPrior(
        name=f'B:{task_type}_{df_task_params["mu"].iloc[1]}_{df_task_params["df"].iloc[1]}_{df_task_params["sigma"].iloc[1]}',
        thetas=thetas,
        distribution=f.get_StudentT_pdf(
            mu=df_task_params["mu"].iloc[1], 
            df=df_task_params["df"].iloc[1], 
            sigma=df_task_params["sigma"].iloc[1],
            thetas=thetas
        )
    )

else:
    raise ValueError(f'Unknown task type: {task_type}')

In [ ]:
# Generative model
# likelihood model
lh_model = GaussianLikelihoodModel(
    thetas=thetas, 
    equivalent_sigma=equivalent_sigma
)

In [ ]:
# likelihood coding information gap calculator
info_gap_calculator_likelihood = LikelihoodCodingInformationGapCalculator(
    lh_model=lh_model,
    prior_A=prior_A,
    prior_B=prior_B,
    p_task_A=p_task_A,
    p_task_B=p_task_B
)

# compute information gap for likelihood coding
expected_posterior_entropy_lh, info_gap_lh = info_gap_calculator_likelihood.compute_information_gap()

In [ ]:
# posterior coding information gap calculator
info_gap_calculator_posterior = PosteriorCodingInformationGapCalculator(
    lh_model=lh_model,
    prior_A=prior_A,
    prior_B=prior_B,
    p_task_A=p_task_A,
    p_task_B=p_task_B
)

# compute information gap for posterior coding
pairs, pairwise_posterior_kl_divergences = info_gap_calculator_posterior._get_pairwise_posterior_kl_divergences()
info_gap_posterior = info_gap_calculator_posterior.compute_information_gap()